In [1]:
import duckdb
import os

# Get Wasabi credentials from environment
wasabi_endpoint = os.getenv('WASABI_ENDPOINT', 's3.us-east-2.wasabisys.com')
wasabi_access_key = os.getenv('WASABI_ACCESS_KEY')
wasabi_secret_key = os.getenv('WASABI_SECRET_KEY')
bucket_name = os.getenv('WASABI_BUCKET_NAME')

# Create DuckDB connection
con = duckdb.connect()

# Configure S3 settings
con.execute(f"""
    SET s3_endpoint='{wasabi_endpoint}';
    SET s3_access_key_id='{wasabi_access_key}';
    SET s3_secret_access_key='{wasabi_secret_key}';
    SET s3_url_style='path';
    SET preserve_insertion_order = false;
    SET enable_progress_bar = true;
""")

In [2]:
sport = 'NFL'
dds_contests_path = f"s3://{bucket_name}/dds/{sport}/contests/*/*/data.parquet"

In [3]:
lineups_path = f"s3://{bucket_name}/dds/{sport}/lineups/*/*/data.parquet"

In [5]:
merged_df = con.execute(
    f"""
    with contests as (
    select contest_id,
        contest_group_id as slate_id,
        cash_line
     from read_parquet('{dds_contests_path}') where is_largest_by_size = TRUE
    ),
    lineups as (
        select
        contest_id,
        min(points) as points,
        max(lineup_rank) as lineup_rank
        from read_parquet('{lineups_path}')
            where is_cashing = TRUE
            group by contest_id
    )
    select distinct
        slate_id,
        contests.contest_id,
        points,
        cash_line,
        lineup_rank
        from contests
            left join lineups on contests.contest_id = lineups.contest_id
"""
).df()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [6]:
merged_df.drop_duplicates()

,slate_id,contest_id,points,cash_line,lineup_rank
0,112655,166468299,123.86,8465,8465
1,100523,154709411,107.56,760,759
2,116997,171974507,74.83,373,372
3,114975,169563863,83.26,51,51
4,99692,153834090,62.80,24885,24872
...,...,...,...,...,...
1237,98607,152446282,137.80,43175,43175
1238,100033,154204370,150.92,117,117
1239,102909,157362518,NaN,26495,<NA>
1240,113845,168118996,NaN,21815,<NA>


In [12]:
contests_df = con.execute(f"""
    select * from read_parquet('{dds_contests_path}') limit 10
""")

In [14]:
contests_df.df()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,contest_id,contest_name,contest_size,entry_cost,total_prizes,multi_entry_max,is_largest_by_size,is_primary,tour,source_id,sport_event_id,contest_group_id,cash_line,date_id
0,149435843,NFL $100K Screen Pass [$25K to 1st] (Thu-Mon),7843,15,100000,150,True,False,,4,0,96716,1710,20230907
1,147325137,NFL $3M MEGA Millionaire [$1M to 1st + ToC Sem...,768,4444,3072667,23,True,True,,4,0,95302,145,20230910
2,150098355,NFL $201K Luxury Box [$75K to 1st],67,3180,201000,2,False,False,,4,0,95302,13,20230910
3,147325161,"NFL $350K Game Changer [$100K to 1st, Single E...",247,1500,350000,1,False,False,,4,0,95302,55,20230910
4,147325138,NFL $750K Wildcat [$150K to 1st],2502,333,750000,75,False,False,,4,0,95302,541,20230910
5,147325090,"NFL $600K Power Sweep [3 Entry Max, $100K to 1st]",4444,150,600000,3,False,False,,4,0,95302,1005,20230910
6,147325139,NFL $2.5M Fantasy Football Millionaire [$1M to...,28029,100,2522667,150,True,True,,4,0,95302,5620,20230910
7,147325101,"NFL $300K End Zone [$50K to 1st, Single Entry]",4545,75,300000,1,False,False,,4,0,95302,1090,20230910
8,147325103,"NFL $200K Red Zone [$25K to 1st, Single Entry]",4545,50,200000,1,False,False,,4,0,95302,999,20230910
9,147325076,NFL $500K Bootleg [$100K to 1st],17669,33,500000,150,False,False,,4,0,95302,4100,20230910


In [15]:
lineups_df = con.execute(f"""select * from read_parquet('{lineups_path}') limit 10""").df()
lineups_df

InvalidInputException: Invalid Input Error: Failed to read file "s3://dfscrunch-data-lake/dds/NFL/lineups/dk_classic/2023-10-05/data.parquet": schema mismatch in glob: column "pos_dst1" was read from the original file "s3://dfscrunch-data-lake/dds/NFL/lineups/dk_classic/2023-09-07/data.parquet", but could not be found in file "s3://dfscrunch-data-lake/dds/NFL/lineups/dk_classic/2023-10-05/data.parquet".
Candidate names: contest_id, lineup_hash, lineup_ct, lineup_user_ct, points, total_salary, total_own, min_own, max_own, avg_own, lineup_rank, is_cashing, favorite_ct, underdog_ct, home_ct, visitor_ct, payout, lineup_percentile, correlated_players, pos_qb1, pos_rb1, pos_rb2, pos_te1, pos_wr1, pos_wr2, pos_wr3, team_stacks, game_stacks, lineup_trends, entry_name_list
If you are trying to read files with different schemas, try setting union_by_name=True